In [1]:
from pprint import PrettyPrinter
import numpy as np
import pandas as pd
from pymongo import MongoClient
import json

In [2]:
# Connect to Mongodb
client = MongoClient('mongodb://localhost:27017')

In [3]:
db = client['riotGamesOlddb']
match_collection = db['matchdatas']

In [4]:
to_get = {'matchSummaryData.info.participants':1, 'matchSummaryData.info.gameStartTimestamp':1, "_id":0,"title":1}
match_cursor = match_collection.find({"$and":[{'matchSummaryData.info.gameMode':"CLASSIC"},{'matchSummaryData.metadata.participants':{'$size':10}}]},to_get)


In [5]:
participant_summary = []
timestamp = []

for match in match_cursor:
    participant_summary.append(match['matchSummaryData']['info']['participants'])
    timestamp.append(match['matchSummaryData']['info']['gameStartTimestamp'])
print(len(participant_summary))
participant_summary[0]

2004


[{'assists': 3,
  'baronKills': 0,
  'basicPings': 1,
  'bountyLevel': 0,
  'challenges': {'12AssistStreakCount': 0,
   'abilityUses': 42,
   'acesBefore15Minutes': 0,
   'alliedJungleMonsterKills': 0,
   'baronTakedowns': 0,
   'blastConeOppositeOpponentCount': 0,
   'bountyGold': 0,
   'buffsStolen': 0,
   'completeSupportQuestInTime': 0,
   'controlWardsPlaced': 0,
   'damagePerMinute': 130.98680645405489,
   'damageTakenOnTeamPercentage': 0.1220907607694343,
   'dancedWithRiftHerald': 0,
   'deathsByEnemyChamps': 8,
   'dodgeSkillShotsSmallWindow': 0,
   'doubleAces': 0,
   'dragonTakedowns': 0,
   'earlyLaningPhaseGoldExpAdvantage': 0,
   'effectiveHealAndShielding': 136,
   'elderDragonKillsWithOpposingSoul': 0,
   'elderDragonMultikills': 0,
   'enemyChampionImmobilizations': 0,
   'enemyJungleMonsterKills': 0,
   'epicMonsterKillsNearEnemyJungler': 0,
   'epicMonsterKillsWithin30SecondsOfSpawn': 0,
   'epicMonsterSteals': 0,
   'epicMonsterStolenWithoutSmite': 0,
   'flawlessAc

Dataframe creation:

In [6]:
p_df = pd.json_normalize(json.loads((pd.DataFrame(participant_summary)).to_json(orient="records")))

p_df.head(5)

,0.assists,0.baronKills,0.basicPings,0.bountyLevel,0.challenges.12AssistStreakCount,0.challenges.abilityUses,0.challenges.acesBefore15Minutes,0.challenges.alliedJungleMonsterKills,0.challenges.baronTakedowns,0.challenges.blastConeOppositeOpponentCount,...,3.challenges.teleportTakedowns,4.challenges.teleportTakedowns,9.challenges.teleportTakedowns,3.challenges.fasterSupportQuestCompletion,0.challenges.mejaisFullStackInTime,4.challenges.mejaisFullStackInTime,5.challenges.mejaisFullStackInTime,9.challenges.mejaisFullStackInTime,3.challenges.mejaisFullStackInTime,8.challenges.mejaisFullStackInTime
0,3,0,1.0,0,0.0,42.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0.0,0,0.0,78.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,0,0.0,0,0.0,194.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,29.0,0,0.0,183.0,0.0,18.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,1.0,4,0.0,100.0,0.0,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Champion Mastery points:

Data Processing - Remove unnecessary columns:

In [7]:
to_remove = ['championName', 'championTransform', 'eligibleForProgression', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'individualPosition', 'lane', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'participantId', 'profileIcon', 'puuid', 'riotIdName', 'riotIdTagline', 'role', 'summonerId', 'teamEarlySurrendered', 'teamId', 'teamPosition', 'unrealKills', 'challenges.blastConeOppositeOpponentCount', 'challenges.completeSupportQuestInTime', 'challenges.controlWardTimeCoverageInRiverOrEnemyHalf', 'challenges.dancedWithRiftHerald', 'challenges.dodgeSkillShotsSmallWindow', 'challenges.doubleAces', 'challenges.elderDragonKillsWithOpposingSoul', 'challenges.epicMonsterKillsNearEnemyJungler', 'challenges.epicMonsterKillsWithin30SecondsOfSpawn', 'challenges.epicMonsterStolenWithoutSmite', 'challenges.firstTurretKilledTime', 'challenges.flawlessAces', 'challenges.hadOpenNexus', 'challenges.immobilizeAndKillWithAlly', 'challenges.initialBuffCount', 'challenges.initialCrabCount', 'challenges.junglerTakedownsNearDamagedEpicMonster', 'challenges.killAfterHiddenWithAlly', 'challenges.killedChampTookFullTeamDamageSurvived', 'challenges.killsNearEnemyTurret', 'challenges.killsOnRecentlyHealedByAramPack', 'challenges.killsUnderOwnTurret', 'challenges.killsWithHelpFromEpicMonster', 'challenges.knockEnemyIntoTeamAndKill', 'challenges.lostAnInhibitor', 'challenges.moreEnemyJungleThanOpponent', 'challenges.multiKillOneSpell', 'challenges.multiTurretRiftHeraldCount', 'challenges.multikillsAfterAggressiveFlash', 'challenges.outerTurretExecutesBefore10Minutes', 'challenges.outnumberedKills', 'challenges.outnumberedNexusKill', 'challenges.perfectDragonSoulsTaken', 'challenges.perfectGame', 'challenges.pickKillWithAlly', 'challenges.playedChampSelectPosition', 'challenges.poroExplosions', 'challenges.quickCleanse', 'challenges.quickFirstTurret', 'challenges.quickSoloKills', 'challenges.saveAllyFromDeath', 'challenges.scuttleCrabKills', 'challenges.snowballsHit', 'challenges.soloBaronKills', 'challenges.survivedSingleDigitHpCount', 'challenges.survivedThreeImmobilizesInFight', 'challenges.takedownOnFirstTurret', 'challenges.takedownsAfterGainingLevelAdvantage', 'challenges.takedownsFirstXMinutes', 'challenges.takedownsInAlcove', 'challenges.takedownsInEnemyFountain', 'challenges.threeWardsOneSweeperCount', 'challenges.tookLargeDamageSurvived', 'challenges.turretsTakenWithRiftHerald', 'challenges.twentyMinionsIn3SecondsCount', 'challenges.unseenRecalls', 'challenges.wardsGuarded', 'perks.statPerks.defense', 'perks.statPerks.flex', 'perks.statPerks.offense', 'perks.styles', 'challenges.earliestDragonTakedown', 'challenges.highestCrowdControlScore', 'challenges.junglerKillsEarlyJungle', 'challenges.highestChampionDamage', 'challenges.fasterSupportQuestCompletion', 'challenges.earliestBaron', 'challenges.highestWardKills','challenges.mejaisFullStackInTime', 'challenges.teleportTakedowns', 'challenges.mostWardsDestroyedOneSweeper', 'challenges.earliestElderDragon', 'challenges.hadAfkTeammate', 'challenges.fastestLegendary', 'challenges.shortestTimeToAceFromFirstTakedown','sightWardsBoughtInGame', 'challenges.takedownsFirst25Minutes', 'challenges.thirdInhibitorDestroyedTime', 'challenges.killsOnLanersEarlyJungleAsJungler', 'challenges.baronBuffGoldAdvantageOverThreshold', 'challenges.12AssistStreakCount', 'challenges.acesBefore15Minutes', 'challenges.alliedJungleMonsterKills', 'challenges.baronTakedowns', 'challenges.buffsStolen',  'challenges.elderDragonMultikills',  'challenges.enemyJungleMonsterKills',  'challenges.killsOnOtherLanesEarlyJungleAsLaner', 'challenges.epicMonsterSteals', 'challenges.getTakedownsInAllLanesEarlyJungleAsLaner', 'challenges.landSkillShotsEarlyGame', 'challenges.teamBaronKills', 'challenges.wardTakedownsBefore20M', 'challenges.killingSprees']

In [8]:
remove_cols = []

for e in to_remove:
    for i in range(10):
        remove_cols.append(str(i)+'.'+e)
len(remove_cols)

1150

In [9]:
p_df = p_df.drop(remove_cols,axis=1)

Data Processing - Remove individual player win and add team win

In [10]:
p_df['teamWon'] = np.where(p_df['0.win'] == 1, 0, 1)

In [11]:
p_df = p_df.drop((p_df.filter(regex='(win)$', axis="columns")),axis=1)

Data Processing - Replace NA with 0:

In [12]:
p_df = p_df.replace(np.nan, 0)

In [13]:
p_df.head(5)

,0.assists,0.baronKills,0.basicPings,0.bountyLevel,0.challenges.abilityUses,0.challenges.bountyGold,0.challenges.controlWardsPlaced,0.challenges.damagePerMinute,0.challenges.damageTakenOnTeamPercentage,0.challenges.deathsByEnemyChamps,...,6.challenges.killParticipation,7.challenges.killParticipation,8.challenges.killParticipation,9.challenges.killParticipation,7.challenges.soloTurretsLategame,1.challenges.soloTurretsLategame,6.challenges.soloTurretsLategame,9.challenges.soloTurretsLategame,4.challenges.soloTurretsLategame,teamWon
0,3,0,1.0,0,42.0,0.0,0.0,130.986806,0.122091,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1,0,0.0,0,78.0,300.0,0.0,348.257720,0.262037,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,8,0,0.0,0,194.0,450.0,0.0,292.073547,0.180031,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,0,29.0,0,183.0,0.0,0.0,401.802877,0.255846,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,4,0,1.0,4,100.0,0.0,0.0,502.155392,0.136445,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Data Processing: Create Summoner codes

Remove summoner name:

In [14]:
p_df = p_df.drop((p_df.filter(regex='(summonerName)$', axis="columns")),axis=1)

Data Processing - Replace bool with int:


In [15]:
p_df = p_df.drop(p_df.select_dtypes('bool'),axis = 1)

In [16]:
p_df.dtypes

0.assists                             int64
0.baronKills                          int64
0.basicPings                        float64
0.bountyLevel                         int64
0.challenges.abilityUses            float64
                                     ...   
1.challenges.soloTurretsLategame    float64
6.challenges.soloTurretsLategame    float64
9.challenges.soloTurretsLategame    float64
4.challenges.soloTurretsLategame    float64
teamWon                               int32
Length: 1191, dtype: object

In [17]:
p_df.astype(np.float32).dtypes

0.assists                           float32
0.baronKills                        float32
0.basicPings                        float32
0.bountyLevel                       float32
0.challenges.abilityUses            float32
                                     ...   
1.challenges.soloTurretsLategame    float32
6.challenges.soloTurretsLategame    float32
9.challenges.soloTurretsLategame    float32
4.challenges.soloTurretsLategame    float32
teamWon                             float32
Length: 1191, dtype: object

Sort the columns

In [18]:
p_df = p_df.reindex(sorted(p_df.columns), axis=1)


Column headers:

In [21]:
single_header = p_df.loc[:, p_df.columns.str.match('^0')].columns.values.tolist()


Export CSV 

In [ ]:
p_df.to_csv('match_dataframe.csv')